In [ ]:
 !pip install langchain
 !pip install openai
 !pip install PyPDF2
 !pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [ ]:
# Get your API keys from openai, you will need to create an account.
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
# location of the pdf file/files.
reader = PdfReader('/content/gdrive/MyDrive/LangChain/yuksekogretim-kurulu-transkript-belgesi-sorgulama.pdf')

In [ ]:
reader

In [ ]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
raw_text

'Öğrenci No\nT.C. Kimlik No\nAdı\nDoğum Tarihi195260303\n46831086028\nONUR\n03/04/1997:\n::\n:(Student ID)\n(TR Identity No)\n(Given Name)\n(Date of Birth)(Surname) : Soyadı SAKAR\n FIRAT ÜNİVERSİTESİ\nNOT DÖKÜM BELGESİ(FIRAT UNIVERSITY)\n(TRANSCRIPT)\n27.09.2022 Belge Tarihi\n(Date of Issue):\n YOKTR70BYXHERRSB5S\nEğitim Birimi:Mühendislik Fakültesi\n(Academic Unit) (Faculty Of Engineering)\nProgramı/ABD/ASD:Bilgisayar Mühendisliği Pr.\n(Program/Department) (Computer Engineering Pr.)\nAkademik Derece Türü:Lisans\n(Type of Academic Degree) (Bachelor`s Degree)\nÖğretim Dili:Türkçe\n(Language of Instruction) (Turkish)\nEğitim ve Öğretim Türü:Örgün Öğretim (İkinci Öğretim)\n(Type of Education) (Formal Education (Evening Education))\nProgram Türü:Anadal\n(Program Type) (Major)\nAktif Öğrenci:Öğrenim Durumu\n(Education Status) (Active Student)\n:ISCED Kodu0714(ISCED Code)09/09/2019:Kayıt Tarihi\n(Admission Date)\n:DGS\n(External Transfer Exam (DGS))Giriş Türü\n(Entry Type)\nSınıfı / Dönemi:

In [ ]:
raw_text[:100]

'Öğrenci No\nT.C. Kimlik No\nAdı\nDoğum Tarihi195260303\n46831086028\nONUR\n03/04/1997:\n::\n:(Student ID)\n(T'

In [ ]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits.

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

16

In [ ]:
texts[0]

'Öğrenci No\nT.C. Kimlik No\nAdı\nDoğum Tarihi195260303\n46831086028\nONUR\n03/04/1997:\n::\n:(Student ID)\n(TR Identity No)\n(Given Name)\n(Date of Birth)(Surname) : Soyadı SAKAR\n FIRAT ÜNİVERSİTESİ\nNOT DÖKÜM BELGESİ(FIRAT UNIVERSITY)\n(TRANSCRIPT)\n27.09.2022 Belge Tarihi\n(Date of Issue):\n YOKTR70BYXHERRSB5S\nEğitim Birimi:Mühendislik Fakültesi\n(Academic Unit) (Faculty Of Engineering)\nProgramı/ABD/ASD:Bilgisayar Mühendisliği Pr.\n(Program/Department) (Computer Engineering Pr.)\nAkademik Derece Türü:Lisans\n(Type of Academic Degree) (Bachelor`s Degree)\nÖğretim Dili:Türkçe\n(Language of Instruction) (Turkish)\nEğitim ve Öğretim Türü:Örgün Öğretim (İkinci Öğretim)\n(Type of Education) (Formal Education (Evening Education))\nProgram Türü:Anadal\n(Program Type) (Major)\nAktif Öğrenci:Öğrenim Durumu\n(Education Status) (Active Student)\n:ISCED Kodu0714(ISCED Code)09/09/2019:Kayıt Tarihi\n(Admission Date)\n:DGS\n(External Transfer Exam (DGS))Giriş Türü\n(Entry Type)\nSınıfı / Dönemi:

In [ ]:
texts[1]

'We collected roughly one million prompt-\nresponse pairs using the GPT-3.5-Turbo OpenAI\nAPI between March 20, 2023 and March 26th,\n2023. To do this, we first gathered a diverse sam-\nple of questions/prompts by leveraging three pub-\nlicly available datasets:\n• The unified chip2 subset of LAION OIG.\n• Coding questions with a random sub-sample\nof Stackoverflow Questions\n• Instruction-tuning with a sub-sample of Big-\nscience/P3\nWe chose to dedicate substantial attention to data\npreparation and curation based on commentary in\nthe Stanford Alpaca project (Taori et al., 2023).\nUpon collection of the initial dataset of prompt-\ngeneration pairs, we loaded data into Atlas for data\ncuration and cleaning. With Atlas, we removed all\nexamples where GPT-3.5-Turbo failed to respond\nto prompts and produced malformed output. This\nreduced our total number of examples to 806,199\nhigh-quality prompt-generation pairs. Next, we\ndecided to remove the entire Bigscience/P3 sub-'

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
docsearch

NameError: ignored

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "who are the authors of the article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

NameError: ignored

In [ ]:
query = "What was the cost of training the GPT4all model?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' $100'

In [ ]:
query = "How was the model trained?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The model was trained with LoRA (Hu et al., 2021) on the 437,605 post-processed examples for four epochs. Detailed model hyper-parameters and training code can be found in the associated repository and model training log.'

In [ ]:
query = "what was the size of the training dataset?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The final training dataset contains 437,605 prompt-generation pairs.'

In [ ]:
query = "How is this different from other models?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' This model is different from other models because it is based on LLaMA, it is licensed only for research purposes, and it is trained on a dataset of post-processed examples. It also has a TSNE visualization of the final training data, and a zoomed-in view to show generations related to personal health and wellness.'

In [ ]:
query = "What is Google Bard?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."